# Custom Metrics

In machine learning, metrics are evaluation measures used to assess the performance and effectiveness of a model. They provide quantitative information about how well the model is performing on a specific task, such as classification, regression, or clustering.

**Loss**:
- Loss is a value that quantifies the error or discrepancy between the predicted output of a model and the true expected output during training.
- The loss function is defined based on the specific task and data, and its purpose is to guide the model's optimization process by minimizing the error.
- During training, the model learns by adjusting its parameters to minimize the loss.

**Metrics**:
- Metrics are evaluation measures that assess the performance and effectiveness of a model on a specific task.
- Metrics provide insights into various aspects of the model's performance, such as accuracy, precision, recall, F1 score, mean squared error, etc.
- While the loss function guides the model's optimization, metrics provide a more comprehensive view of the model's performance from a user's perspective.

In summary, loss is an optimization criterion used during training to adjust the model's parameters, while metrics are evaluation measures used to assess the model's performance on unseen data or in real-world scenarios.

### By Subclassing
- If you need a metric that isn't part of the API, you can easily create custom metrics by subclassing the **tf.keras.metrics.Metric** class.
- You will need to implement 4 methods:
   - **\_\_init\_\_**(self), in which you will create state variables for your metric.
   - **update_state(self, y_true, y_pred, sample_weight=None)**, which uses the targets y_true and the model predictions y_pred to update the state variables.
   - **result(self)**, which uses the state variables to compute the final results.
   - **reset_state(self)**, which reinitializes the state of the metric.

- **State update** and **results computation** are kept separate (in update_state() and result(), respectively) because in some cases, the results computation might be very expensive and would only be done periodically.

In [15]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

Here's a simple example showing how to implement a CategoricalTruePositives metric that counts how many samples were correctly classified as belonging to a given class

**Note**: In the custom metric example provided, the use of **add_weight** is not mandatory, but it is a recommended approach to define and track the metric's internal variables. By using add_weight, the internal variables are automatically *tracked* by Keras and can be *accessed*, *initialized*, and *updated* within the metric class.

In [76]:
class CategoricalTruePositives(keras.metrics.Metric):
    def __init__(self, name="categorical_true_positive", **kwargs):
        super().__init__(name=name, **kwargs)
        self.true_positive = self.add_weight(name="ctp", initializer="zeros")
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.reshape(tf.argmax(y_pred, axis=1), shape=(-1, 1))
        values = tf.math.equal(tf.cast(y_true, "int32"), tf.cast(y_pred, "int32"))
        values = tf.cast(values, "float32")
        if sample_weight:
            sample_weight = tf.cast(sample_weight, "float32")
            values = tf.multiply(values, sample_weight)
        self.true_positive.assign_add(tf.reduce_sum(values))
    def result(self):
        return self.true_positive
    def reset_state(self):
        self.true_positive.assign(0.0)

In [77]:
model = keras.Sequential([
    layers.Input((10)),
    layers.Dense(10, activation="relu"),
    layers.Dense(4, activation="sigmoid")
])
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 10)                110       
                                                                 
 dense_17 (Dense)            (None, 4)                 44        
                                                                 
Total params: 154
Trainable params: 154
Non-trainable params: 0
_________________________________________________________________


In [78]:
x = tf.random.normal([10, 10])
y_true = tf.math.round(tf.random.uniform((10, 1), minval=0, maxval=3))
x.shape, y_true.shape, y_true

(TensorShape([10, 10]),
 TensorShape([10, 1]),
 <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[1.],
        [3.],
        [1.],
        [1.],
        [1.],
        [0.],
        [2.],
        [1.],
        [2.],
        [0.]], dtype=float32)>)

In [79]:
model.compile(optimizer="adam",
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=[CategoricalTruePositives()])

In [80]:
model.fit(x, y_true, epochs=10)

Epoch 1/10
1/1 [==============================] - 0s 283ms/step - loss: 1.7789 - categorical_true_positive: 2.0000
Epoch 2/10
1/1 [==============================] - 0s 5ms/step - loss: 1.7640 - categorical_true_positive: 3.0000
Epoch 3/10
1/1 [==============================] - 0s 6ms/step - loss: 1.7491 - categorical_true_positive: 3.0000
Epoch 4/10
1/1 [==============================] - 0s 5ms/step - loss: 1.7343 - categorical_true_positive: 3.0000
Epoch 5/10
1/1 [==============================] - 0s 5ms/step - loss: 1.7196 - categorical_true_positive: 3.0000
Epoch 6/10
1/1 [==============================] - 0s 4ms/step - loss: 1.7051 - categorical_true_positive: 3.0000
Epoch 7/10
1/1 [==============================] - 0s 8ms/step - loss: 1.6906 - categorical_true_positive: 3.0000
Epoch 8/10
1/1 [==============================] - 0s 4ms/step - loss: 1.6763 - categorical_true_positive: 3.0000
Epoch 9/10
1/1 [==============================] - 0s 9ms/step - loss: 1.6621 - categorical_tru

# Handling losses and metrics that doesnot fit the standard signature
[more](https://www.tensorflow.org/guide/keras/training_with_built_in_methods#handling_losses_and_metrics_that_dont_fit_the_standard_signature)

- The overwhelming majority of losses and metrics can be computed from y_true and y_pred, where y_pred is an output of your model -- but not all of them. For instance, a regularization loss may only require the activation of a layer (there are no targets in this case), and this activation may not be a model output.